<a href="https://colab.research.google.com/github/deveworld/Gemago/blob/main/Gemago_7b_Infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers bitsandbytes accelerate

In [ ]:
!pip install flash-attn --no-build-isolation

# Load the Gemago 7b Model from HF

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained("devworld/gemago-7b")

# Use 4bit quantization_config
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(
    "devworld/gemago-7b",
    device_map="auto",
    quantization_config=quantization_config,
    # attn_implementation="flash_attention_2" # Not in T4
  )

In [33]:
def gen(text, max_length):
    input_ids = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **input_ids,
        max_length=max_length,
        temperature=0.3,
        top_p=0.8,
        do_sample=True,
    )
    return tokenizer.decode(outputs[0])

def e2k(e):
    res = []
    for input_e in e.split("\n"):
        input_text = f"English:\n{input_e}\n\nKorean:\n"
        res.append(gen(input_text, 2048))
    return res

def k2e(k):
    res = []
    for input_e in k.split("\n"):
        input_text = f"Korean:\n{input_e}\n\nEnglish:\n"
        res.append(gen(input_text, 2048))
    return res

# Korean to English Demo

In [ ]:
k2e("나라의 말이 중국과 달라 문자와 서로 통하지 아니하므로, 이런 까닭으로 어리석은 백성이 이르고자 하는 바가 있어도 마침내 제 뜻을 능히 펴지 못하는 사람이 많으니라. 내가 이를 위하여 가엾게 여겨 새로 스물여덟 자를 만드니 사람마다 하여금 쉽게 익혀 날로 쓰기 편하게 하고자 할 따름이니라.")

# English to Korean Demo

In [ ]:
e2k("Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.")